In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import datetime as dt
import gc
from sklearn.ensemble import RandomForestClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from lightgbm import LGBMClassifier
%matplotlib inline
start_time = dt.datetime.now()

In [61]:
df_1=pd.read_csv('create_vars.csv', index_col=0)
df_1

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,Amount_state_count_1_by_30,Amount_zip_count_0_by_7,Amount_zip_count_0_by_14,Amount_zip_count_0_by_30,Amount_zip_count_1_by_7,Amount_zip_count_1_by_14,Amount_zip_count_1_by_30,Cardnum_U*,Merch_U*,Risk
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,2.178008,NaN,0.02242
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,1.604857,1.000894,0.02242
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,2.368143,1.025818,0.02242
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,1.044105,NaN,0.02242
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,2.178008,NaN,0.02242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96392,96749,5142276053,2006-12-31,3500000006160,BEST BUY 00001610,KY,41042.0,P,84.79,0,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,1.002393,1.000894,0.02242
96393,96750,5142225701,2006-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248.0,P,118.75,0,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,1.137948,1.288057,0.02242
96394,96751,5142226486,2006-12-31,4503057341100,"TECH PAC, INC",OH,45150.0,P,363.56,0,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,1.201338,1.106055,0.02242
96395,96752,5142244619,2006-12-31,8834000695412,BUY.COM,CA,92656.0,P,2202.03,0,...,0.033333,0.142857,0.071429,0.033333,0.142857,0.071429,0.033333,1.499767,1.029441,0.02242


In [62]:
df_2=pd.read_csv('final_vars_list.csv')
df_2['variable name'].to_list()

['card_zip_total_3',
 'card_state_max_30',
 'card_merch_total_14',
 'card_merch_total_0',
 'card_state_total_0',
 'card_zip_total_0',
 'card_zip_max_30',
 'card_state_total_30',
 'card_state_avg_7',
 'card_zip_avg_3',
 'card_merch_total_30',
 'card_state_total_7',
 'card_zip_avg_14',
 'card_zip_max_14',
 'card_merch_avg_7',
 'card_merch_avg_3',
 'card_merch_avg_14',
 'card_merch_max_30',
 'card_zip_avg_7',
 'card_state_max_0']

In [75]:
cardnum_df=df_1[['Cardnum', 'Date','Fraud', 'card_zip_total_3',
 'card_state_max_30',
 'card_merch_total_14',
 'card_merch_total_0',
 'card_state_total_0',
 'card_zip_total_0',
 'card_zip_max_30',
 'card_state_total_30',
 'card_state_avg_7',
 'card_zip_avg_3',
 'card_merch_total_30',
 'card_state_total_7',
 'card_zip_avg_14',
 'card_zip_max_14',
 'card_merch_avg_7',
 'card_merch_avg_3',
 'card_merch_avg_14',
 'card_merch_max_30',
 'card_zip_avg_7',
 'card_state_max_0']]
cardnum_df

,Cardnum,Date,Fraud,card_zip_total_3,card_state_max_30,card_merch_total_14,card_merch_total_0,card_state_total_0,card_zip_total_0,card_zip_max_30,...,card_merch_total_30,card_state_total_7,card_zip_avg_14,card_zip_max_14,card_merch_avg_7,card_merch_avg_3,card_merch_avg_14,card_merch_max_30,card_zip_avg_7,card_state_max_0
0,5142190439,2006-01-01,0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62
1,5142183973,2006-01-01,0,31.42,31.42,31.42,31.42,31.42,31.42,31.42,...,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42
2,5142131721,2006-01-01,0,178.49,178.49,178.49,178.49,178.49,178.49,178.49,...,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49
3,5142148452,2006-01-01,0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62
4,5142190439,2006-01-01,0,7.24,3.62,7.24,7.24,7.24,7.24,3.62,...,7.24,7.24,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96392,5142276053,2006-12-31,0,84.79,84.79,84.79,84.79,84.79,84.79,84.79,...,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79
96393,5142225701,2006-12-31,0,118.75,1385.00,718.60,118.75,118.75,118.75,599.85,...,718.60,886.25,359.30,599.85,118.75,118.75,359.30,599.85,118.75,118.75
96394,5142226486,2006-12-31,0,363.56,363.56,363.56,363.56,363.56,363.56,363.56,...,363.56,435.41,363.56,363.56,363.56,363.56,363.56,363.56,363.56,363.56
96395,5142244619,2006-12-31,0,2202.03,2202.03,2202.03,2202.03,2202.03,2202.03,2202.03,...,2745.63,2202.03,2202.03,2202.03,2202.03,2202.03,2202.03,2202.03,2202.03,2202.03


In [76]:
cardnum_df.groupby('Cardnum').size().sort_values(ascending=False).head(15)

Cardnum
5142148452    1192
5142184598     921
5142189108     663
5142297710     583
5142223373     579
5142187452     526
5142299634     515
5142189945     512
5142149691     497
5142190147     488
5142230181     479
5142157720     475
5142152067     473
5142143744     422
5142125684     415
dtype: int64

In [77]:
cardnum_df=cardnum_df[(cardnum_df['Cardnum']== 5142148452)]
cardnum_df

,Cardnum,Date,Fraud,card_zip_total_3,card_state_max_30,card_merch_total_14,card_merch_total_0,card_state_total_0,card_zip_total_0,card_zip_max_30,...,card_merch_total_30,card_state_total_7,card_zip_avg_14,card_zip_max_14,card_merch_avg_7,card_merch_avg_3,card_merch_avg_14,card_merch_max_30,card_zip_avg_7,card_state_max_0
3,5142148452,2006-01-01,0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,3.620000,3.62,3.620000,3.620000,3.620000,3.62,3.620000,3.62
15,5142148452,2006-01-01,0,7.36,3.74,7.36,7.36,7.36,7.36,3.74,...,7.36,7.36,3.680000,3.74,3.680000,3.680000,3.680000,3.74,3.680000,3.74
16,5142148452,2006-01-01,0,11.16,3.80,11.16,11.16,11.16,11.16,3.80,...,11.16,11.16,3.720000,3.80,3.720000,3.720000,3.720000,3.80,3.720000,3.80
29,5142148452,2006-01-01,0,14.96,3.80,14.96,14.96,14.96,14.96,3.80,...,14.96,14.96,3.740000,3.80,3.740000,3.740000,3.740000,3.80,3.740000,3.80
36,5142148452,2006-01-01,0,18.70,3.80,18.70,18.70,18.70,18.70,3.80,...,18.70,18.70,3.740000,3.80,3.740000,3.740000,3.740000,3.80,3.740000,3.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95603,5142148452,2006-12-27,0,30.63,38.75,340.48,30.63,30.63,30.63,38.75,...,419.71,271.91,6.547692,38.75,6.323488,3.828750,6.547692,38.75,6.323488,4.37
95617,5142148452,2006-12-27,0,34.43,38.75,344.28,34.43,34.43,34.43,38.75,...,423.51,275.71,6.495849,38.75,6.266136,3.825556,6.495849,38.75,6.266136,4.37
95747,5142148452,2006-12-28,0,45.30,38.75,336.22,10.87,10.87,10.87,38.75,...,434.38,279.84,6.592549,38.75,6.360000,4.530000,6.592549,38.75,6.360000,10.87
95775,5142148452,2006-12-28,0,48.92,38.75,339.84,14.49,14.49,14.49,38.75,...,438.00,283.46,6.535385,38.75,6.299111,4.447273,6.535385,38.75,6.299111,10.87


In [72]:
y=cardnum_df['Fraud']
x=cardnum_df[['card_zip_total_3',
 'card_state_max_30',
 'card_merch_total_14',
 'card_merch_total_0',
 'card_state_total_0',
 'card_zip_total_0',
 'card_zip_max_30',
 'card_state_total_30',
 'card_state_avg_7',
 'card_zip_avg_3',
 'card_merch_total_30',
 'card_state_total_7',
 'card_zip_avg_14',
 'card_zip_max_14',
 'card_merch_avg_7',
 'card_merch_avg_3',
 'card_merch_avg_14',
 'card_merch_max_30',
 'card_zip_avg_7',
 'card_state_max_0']]
rf_model = RandomForestClassifier(n_estimators=120,criterion="gini",max_depth=14,  min_samples_leaf=70,min_samples_split=10,max_features='auto')
rf_model.fit(x,y)

RandomForestClassifier(max_depth=14, min_samples_leaf=70, min_samples_split=10,
                       n_estimators=120)

In [80]:
from sklearn.model_selection import train_test_split
X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst, Y_trntst, test_size = .3)

# here's where you put your final model of choice
model_final = RandomForestClassifier(n_estimators=120,criterion="gini",max_depth=14,  min_samples_leaf=70,min_samples_split=10,max_features='auto')

X_oot = X_oot_orig.copy()
X_trn_save = X_trn.copy()
Y_trn_save = Y_trn.copy()

model_final.fit(X_trn, Y_trn.values.ravel())

NameError: name 'X_trntst' is not defined

In [ ]:
predictions = model_final.predict_proba(x)
8665767575- freedom chase unlimited  4266841534335250 03161966 11/2026 051